In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
file=r'G:/EdData/loans data.csv'

In [3]:
import pandas as pd

In [4]:
from sklearn.ensemble import RandomForestRegressor

In [5]:
data=pd.read_csv(file)

In [6]:
data.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,81174.0,20000,20000,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14,14272,2.0,< 1 year
1,99592.0,19200,19200,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12,11140,1.0,2 years
2,80059.0,35000,35000,21.98%,60 months,debt_consolidation,23.81%,CA,MORTGAGE,11500.00,690-694,14,21977,1.0,2 years
3,15825.0,10000,9975,9.99%,36 months,debt_consolidation,14.30%,KS,MORTGAGE,3833.33,695-699,10,9346,0.0,5 years
4,33182.0,12000,12000,11.71%,36 months,credit_card,18.78%,NJ,RENT,3195.00,695-699,11,14469,0.0,9 years


In [7]:
y=data['Interest.Rate'].str.replace('%','').astype(float)

In [8]:
x=data[['Amount.Requested','FICO.Range']]

In [9]:
temp=x['FICO.Range'].str.split('-',expand=True).astype(int)

In [10]:
x['fico']=(temp[0]+temp[1])/2

In [11]:
del temp
del x['FICO.Range']

In [12]:
x['Amount.Requested']=pd.to_numeric(x['Amount.Requested'],errors='coerce')

In [13]:
rf=RandomForestRegressor(n_estimators=150,max_features=2,bootstrap=True)

In [14]:
for col in x:
    x[col].fillna(x[col].mean(),inplace=True)

In [15]:
x.describe()

,Amount.Requested,fico
count,2500.000000,2500.000000
mean,12412.414830,707.888000
std,7797.762818,35.033161
min,1000.000000,642.000000
25%,6000.000000,682.000000
50%,10000.000000,702.000000
75%,17000.000000,727.000000
max,35000.000000,832.000000


In [16]:
rf.fit(x,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=2, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=150, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [34]:
# lets say we want to find out for what value of fico interest rate will be minimum if amount requested 
# is 5000

# for implementing a GA we'll need to implement some version of crossover and mutation 

# we'll sample 10 random values in the range of fico to start with use their average as the value which gives us optimal point 
# a list of 10 [it could be any other number ], will help us in implementing cross over and mutation
amount_requested=5000

In [17]:
def objective(values):
    
    data=[amount_requested,np.array(values).mean()]
    df=pd.DataFrame(data).T
    pred=rf.predict(df)[0]
    return(pred)

In [18]:
import numpy as np

In [21]:
def obj_pop(population):
    scores=[]
    for values in population:
        scores.append([values,objective(values)])
    return(scores)

In [22]:
def select_top_6(population):
    scores=obj_pop(population)
#     print(scores)
    scores.sort(key=lambda x:x[1])
#     print(scores)
    selected=scores[:6]
    return(selected)

In [29]:
def crossover_mutate(parents):
    pairs=[(0,5),(1,4),(2,3)]
    # this can be made random
    # every pair of parents makes a child , half of child's gene/values
    # come from one parent and rest come from another 
    # then some random values in child are mutated based on some probability threshold 
    # we have taken that to be 0.7
    children=[]
    data=parents.copy()
    for i,j in pairs:
        rand_ind=np.random.choice(range(10),5,replace=False)
        
        child=parents[i].copy()
        for ind in rand_ind:
            child[ind]=parents[j][ind]
        for ind2 in range(len(child)): # mutation
            if np.random.random()>0.7:
                child[ind2]=np.random.randint(500,850,1)
        children.append(child)
    return(children)
    

In [30]:
# we are starting with totally random values in the range
population=np.random.randint(500,850,size=(10,10))

In [31]:
population

array([[849, 734, 572, 737, 830, 811, 676, 593, 728, 614],
       [517, 741, 782, 706, 688, 632, 642, 670, 629, 604],
       [505, 570, 685, 785, 651, 768, 749, 650, 707, 616],
       [671, 565, 845, 762, 848, 768, 806, 731, 556, 691],
       [768, 593, 504, 772, 514, 554, 775, 596, 760, 578],
       [750, 722, 639, 591, 634, 641, 778, 551, 604, 759],
       [765, 567, 647, 679, 541, 839, 825, 833, 720, 723],
       [835, 552, 535, 611, 572, 704, 510, 657, 775, 833],
       [753, 753, 792, 764, 748, 741, 522, 734, 786, 700],
       [727, 705, 769, 519, 520, 735, 644, 660, 560, 582]])

In [32]:
num_gen=100

In [35]:
for _ in range(num_gen):
    scored_pop=select_top_6(population)
    best_score=scored_pop[0][1]
    print(best_score)
    parents=[x for x,_ in select_top_6(population)]
    
    # new population will be top 6 parents + 3 children and + 3 random member
    # as the new population evolves , we will reach to best possible outcome
    
    new_population=parents+crossover_mutate(parents)
    new_population.extend(np.random.randint(500,850,size=(3,10)))
    
    population=new_population


8.179944708994709
8.179944708994709
8.179944708994709
7.191231636141635
7.191231636141635
6.572468888888877
6.572468888888877
6.572468888888877
6.572468888888877
6.572468888888877
6.572468888888877
6.572468888888877
6.572468888888877
6.228208888888884
6.228208888888884
6.228208888888884
6.228208888888884
6.162199999999983
6.162199999999983
6.162199999999983
6.162199999999983
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.118542222222216
6.11854222

In [36]:
best_value=select_top_6(population)[0][0].mean()

In [37]:
best_value

770.4